In [1]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import random

In [17]:
input_path = 'sto/'
kg = input_path + 'sto-enriched.nt'
dataset = pd.read_csv(kg, delimiter='---', header=None)
dataset.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,0
0,<http://www.w3.org/1999/02/22-rdf-syntax-ns#va...
1,<http://www.w3.org/2001/XMLSchema#gYear> <http...
2,<http://www.w3.org/2001/XMLSchema#date> <http:...
3,<http://purl.org/dc/elements/1.1/relation> <ht...
4,<http://purl.org/dc/elements/1.1/description> ...


In [18]:
g = Graph()
g.parse("sto/sto-enriched.nt", format="nt")
    
qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select ?s where {
            ?s rdf:type sto:Standard .
    } limit 1000""")
        
entities = pd.DataFrame(columns=['s'])
obj = []
for row in qres:
    obj.append(str("%s" %row))
entities.s=obj

In [19]:
count = 0
list_pd = pd.DataFrame(columns=['s', 'p', 'o'])
for i in range(entities.shape[0]):
    standard = entities.s[i]
    qres = g.query(
        """PREFIX owl: <http://www.w3.org/2002/07/owl#>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX sto: <https://w3id.org/i40/sto#>
            select distinct ?o where {
                <""" +standard+ """> sto:relatedTo ?o .
            }""")
    lst = pd.DataFrame(columns=['s', 'p', 'o'])
    obj = []
    for row in qres:
        obj.append('<'+str("%s" %row)+'>')
    
    if len(obj)>0:
        lst.o=obj
        lst.s='<'+standard+'>'
        lst.p='<https://w3id.org/i40/sto#relatedTo>'
        list_pd = pd.concat([list_pd, lst], ignore_index=True)
    else:
        count+=1

#list_pd = list_pd.drop_duplicates().reset_index(drop=True)

In [20]:
n_select= round(list_pd.shape[0]/10)
print(n_select)
index = list_pd.index
sampling = random.choices(index, k=n_select)

65


In [21]:
test_set=list_pd.iloc[sampling]
duplicateRowsDF = test_set[test_set.duplicated()]
test_set = test_set.drop_duplicates().reset_index(drop=True)
print(test_set.shape[0])
duplicateRowsDF

61


,s,p,o
321,<https://w3id.org/i40/sto#IEC_62541>,<https://w3id.org/i40/sto#relatedTo>,<https://w3id.org/i40/sto#ISO_22400>
547,<https://w3id.org/i40/sto#ISO_PAS_17506>,<https://w3id.org/i40/sto#relatedTo>,<https://w3id.org/i40/sto#IEC_62424>
165,<https://w3id.org/i40/sto#IEC_62337>,<https://w3id.org/i40/sto#relatedTo>,<https://w3id.org/i40/sto#IEC_62237>
4,<https://w3id.org/i40/sto#ISO_13849>,<https://w3id.org/i40/sto#relatedTo>,<https://w3id.org/i40/sto#IEC_62061>


In [ ]:
#test_set.sort_values(by='s', ascending=False)

In [22]:
test_set['dot'] = '.'

In [23]:
test_set_nt=pd.DataFrame(test_set['s'] + ' '+ test_set['p']+ ' '+ test_set['o']+ ' '+ test_set['dot'])
test_set_nt.head(20)

,0
0,<https://w3id.org/i40/sto#ISO_20922> <https://...
1,<https://w3id.org/i40/sto#ISO_15926> <https://...
2,<https://w3id.org/i40/sto#ISO_PAS_17506> <http...
3,<https://w3id.org/i40/sto#MathML> <https://w3i...
4,<https://w3id.org/i40/sto#IEC_61360> <https://...
5,<https://w3id.org/i40/sto#ISO_PAS_17506> <http...
6,<https://w3id.org/i40/sto#ISO_15926> <https://...
7,<https://w3id.org/i40/sto#eClass> <https://w3i...
8,<https://w3id.org/i40/sto#IEC_61511> <https://...
9,<https://w3id.org/i40/sto#DIN_SPEC_16592> <htt...


In [24]:
training_set = dataset[~dataset.apply(tuple,1).isin(test_set_nt.apply(tuple,1))]
training_set.shape[0]

21088

In [25]:
dataset.shape[0]-training_set.shape[0]

61

In [26]:
training_set.to_csv(input_path+'training_set_relatedTo.csv', index=None)

In [ ]:
training_set.serialize(destination=input_path + 'yago_dataset_person.nt', format='nt')

In [16]:

g = Graph()
g.parse("sto/training_set_relatedTo.nt", format="nt")
    
qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select ?s where {
            ?s rdf:type sto:Standard .
    } limit 1000""")
        
entities = pd.DataFrame(columns=['s'])
obj = []
for row in qres:
    obj.append(str("%s" %row))
entities.s=obj

ParseError: Invalid line: '@en .'

In [ ]:
dataset = pd.DataFrame(columns=['s', 'p', 'o'])
qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX sto: <https://w3id.org/i40/sto#>
        select distinct ?s ?p ?o where {
            ?s ?p ?o .
        }""")
sub = []
prop = []
obj = []
for row in qres:
    if str("%s" %row['o']) =='Nce48e003229e4715a90d165bf90f04bc':
        print((row['p']))
    sub.append('<'+str("%s" %row['s'])+'>')
    prop.append('<'+str("%s" %row['p'])+'>')
    obj.append('<'+str("%s" %row['o'])+'>')
dataset.s=sub
dataset.p=prop
dataset.o=obj

In [ ]:
duplicateRowsDF = dataset[dataset.duplicated()]
dataset = dataset.drop_duplicates().reset_index(drop=True)
print(dataset.shape[0])
duplicateRowsDF

In [ ]:
trainig_set = dataset[~dataset.apply(tuple,1).isin(test_set.apply(tuple,1))]